# Pandas对象简介

如果从底层视角观察 `Pandas` 对象，可以把它们看成增强版的 `NumPy` 结构化数组，行列都不再只是简单的整数索引，还可以带上标签。  
在本章后面的内容中我们将会发现，虽然 `Pandas` 在基本数据结构上实现了许多便利的工具、方法和功能，但是后面将要介绍的每一个工具、方法和功能几乎都需要我们理解基本数据结构的内部细节。  
因此，在深入学习 `Pandas` 之前，先来看看 `Pandas` 的三个基本数据结构：`Series`、`DataFrame` 和 `Index`。

In [1]:
import numpy as np
import pandas as pd

## 1. Pandas的Series对象

`Pandas` 的 `Series` 对象是一个带索引数据构成的一维数组。
`Pandas` 的 `Series` 对象比它模仿的一维 `NumPy` 数组更加通用、灵活。

In [2]:
data = pd.Series([0.25, 0.5, 0.75, 1.0])
data
# 可以看出 Series 对象将一组数据和一组索引绑定在一起，我们可以通过 values 属性和 index 属性获取数据。

0    0.25
1    0.50
2    0.75
3    1.00
dtype: float64

In [3]:
data.values

array([0.25, 0.5 , 0.75, 1.  ])

In [4]:
data.index

RangeIndex(start=0, stop=4, step=1)

In [5]:
data[1]

0.5

In [6]:
data[1:3]

1    0.50
2    0.75
dtype: float64

### 1.1. Series是通用的NumPy数组

`Series` 对象和一维 `NumPy` 数组基本可以等价交换，但两者间的本质差异其实是索引：  
`NumPy` 数组通过隐式定义的整数索引获取数值，而 `Pandas` 的 `Series` 对象用一种显式定义的索引与数值关联。  
显式索引的定义让 `Series` 对象拥有了更强的能力。例如，索引不再仅仅是整数，还可以是任意想要的类型。如果需要，完全可以用字符串定义索引：

In [7]:
data = pd.Series([0.25, 0.5, 0.75, 1.0], index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [8]:
data['b']

0.5

In [9]:
data = pd.Series([0.25, 0.5, 0.75, 1.0], index=[2, 5, 3, 7])
data

2    0.25
5    0.50
3    0.75
7    1.00
dtype: float64

In [10]:
data[5]

0.5

### 1.2. Series是特殊的字典

你可以把 `Pandas` 的 `Series` 对象看成一种特殊的 `Python` 字典。  
字典是一种将任意键映射到一组任意值的数据结构，而 `Series` 对象其实是一种将类型键映射到一组类型值的数据结构。  
类型至关重要：就像 `NumPy` 数组背后特定类型的经过编译的代码使得它在某些操作上比普通的 `Python` 列表更加高效一样，`Pandas Series` 的类型信息使得它在某些操作上比 `Python` 的字典更高效。  
我们可以直接用 `Python` 的字典创建一个 `Series` 对象，让 `Series`对象与字典的类比更加清晰：

In [11]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)  # 用字典创建 Series 对象时，其索引默认按照顺序排列。典型的字典数值获取方式仍然有效：
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [12]:
population['California']

38332521

In [13]:
population['California':'Illinois']  # 和字典不同，Series 对象还支持数组形式的操作，比如切片：

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

### 1.3. 创建Series对象

创建 `Pandas` 的 `Series` 对象的方法，都是像这样的形式：

```python
pd.Series(data, index=index)
```

其中，`index` 是一个可选参数，`data` 参数支持多种数据类型。

In [14]:
pd.Series([2, 4, 6])                 # data 可以是列表或 NumPy 数组，这时 index 默认值为整数序列

0    2
1    4
2    6
dtype: int64

In [15]:
pd.Series(5, index=[100, 200, 300])  # data 也可以是一个标量，创建 Series 对象时会重复填充到每个索引上

100    5
200    5
300    5
dtype: int64

In [16]:
pd.Series({2:'a', 1:'b', 3:'c'})     # data 还可以是一个字典，index 默认是排序的字典键

2    a
1    b
3    c
dtype: object

In [17]:
pd.Series({2:'a', 1:'b', 3:'c'}, index=[3, 2])  # 每一种形式都可以通过显式指定索引筛选需要的结果

3    c
2    a
dtype: object

## 2. Pandas的DataFrame对象

`Pandas` 的另一个基础数据结构是 `DataFrame`。  
和上一节介绍的 `Series`对象一样，`DataFrame` 既可以作为一个通用型 `NumPy` 数组，也可以看作特殊的 `Python` 字典。

### 2.1. DataFrame是通用的NumPy数组

如果将 `Series` 类比为带灵活索引的一维数组，那么 `DataFrame`就可以看作是一种既有灵活的行索引，又有灵活列名的二维数组。  
就像你可以把二维数组看成是有序排列的一维数组一样，你也可以把 `DataFrame` 看成是有序排列的若干 `Series` 对象。  
这里的“排列”指的是它们拥有共同的索引。

`DataFrame` 可以看作一种通用的 `NumPy` 二维数组，它的行与列都可以通过索引获取。

In [18]:
area_dict = {'California': 423967, 'Texas': 695662, 'New York': 141297, 'Florida': 170312, 'Illinois': 149995}
area = pd.Series(area_dict)
area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
dtype: int64

In [19]:
states = pd.DataFrame({'population': population, 'area': area})
states

,population,area
California,38332521,423967
Texas,26448193,695662
New York,19651127,141297
Florida,19552860,170312
Illinois,12882135,149995


In [20]:
states.index

Index(['California', 'Texas', 'New York', 'Florida', 'Illinois'], dtype='object')

In [21]:
states.columns

Index(['population', 'area'], dtype='object')

### 2.2. DataFrame是特殊的字典

与 `Series` 类似，我们也可以把 `DataFrame` 看成一种特殊的字典。字典是一个键映射一个值，而 `DataFrame` 是一列映射一个 `Series` 的数据。

In [22]:
states['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

这里需要注意的是，在 `NumPy` 的二维数组里，`data[0]` 返回第一行；而在 `DataFrame` 中，`data['col0']` 返回第一列。  
因此，最好把 `DataFrame` 看成一种通用字典，而不是通用数组，即使这两种看法在不同情况下都是有用的。

### 2.3. 创建DataFrame对象

In [23]:
pd.DataFrame(population, columns=['population'])  # 通过单个 Series 对象创建。DataFrame 是一组 Series 对象的集合，可以用单个 Series 创建一个单列的 DataFrame：

,population
California,38332521
Texas,26448193
New York,19651127
Florida,19552860
Illinois,12882135


In [24]:
data = [{'a':i, 'b':2*i} for i in range(3)]
pd.DataFrame(data)  # 通过字典列表创建。任何元素是字典的列表都可以变成DataFrame。

,a,b
0,0,0
1,1,2
2,2,4


In [25]:
pd.DataFrame([{'a':1, 'b':2}, {'b':3, 'c':4}])  # 即使字典中有些键不存在，Pandas 也会用缺失值 NaN（不是数字，not a number）来表示：

,a,b,c
0,1.0,2,NaN
1,NaN,3,4.0


In [26]:
pd.DataFrame({'population':population, 'area':area})  # 通过 Series 对象字典创建。就像之前见过的那样，DataFrame也可以用一个由 Series 对象构成的字典创建：

,population,area
California,38332521,423967
Texas,26448193,695662
New York,19651127,141297
Florida,19552860,170312
Illinois,12882135,149995


In [27]:
# 通过 NumPy 二维数组创建。假如有一个二维数组，就可以创建一个可以指定行列索引值的 DataFrame。如果不指定行列索引值，那么行列默认都是整数索引值：
pd.DataFrame(np.random.rand(3,2), columns=['foo', 'bar'], index=['a', 'b', 'c'])

,foo,bar
a,0.495183,0.892617
b,0.838996,0.579059
c,0.539437,0.161771


In [28]:
A = np.zeros(3, dtype=[('A', 'i8'), ('B', 'f8')])
A

array([(0, 0.), (0, 0.), (0, 0.)], dtype=[('A', '<i8'), ('B', '<f8')])

In [29]:
pd.DataFrame(A)  # 通过 NumPy 结构化数组创建。由于 Pandas 的 DataFrame 与结构化数组十分相似，因此可以通过结构化数组创建 DataFrame：

,A,B
0,0,0.0
1,0,0.0
2,0,0.0


## 3. Pandas的Index对象

我们已经发现，`Series` 和 `DataFrame` 对象都使用便于引用和调整的显式索引。  
`Pandas` 的 `Index` 对象是一个很有趣的数据结构，可以将它看作是一个不可变数组或有序集合（实际上是一个多集，因为 `Index` 对象可能会包含重复值）。  
这两种观点使得 `Index` 对象能呈现一些有趣的功能。  
让我们用一个简单的整数列表来创建一个 `Index` 对象：

In [30]:
ind = pd.Index([2, 3, 5, 7, 11])
ind

Int64Index([2, 3, 5, 7, 11], dtype='int64')

### 3.1. 将Index看做不可变数组

`Index` 对象的不可变特征使得多个 `DataFrame` 和数组之间进行索引共享时更加安全，尤其是可以避免因修改索引时粗心大意而导致的副作用。

`Index` 对象的许多操作都像数组。例如，可以通过标准 `Python` 的取值方法获取数值，也可以通过切片获取数值：

In [31]:
ind[1]

3

In [32]:
ind[::2]

Int64Index([2, 5, 11], dtype='int64')

In [33]:
print(ind.size, ind.shape, ind.ndim, ind.dtype)  # Index 对象还有许多与 NumPy 数组相似的属性：

5 (5,) 1 int64


In [34]:
ind[1] = 0  # Index 对象与 NumPy 数组之间的不同在于，Index 对象的索引是不可变的，也就是说不能通过通常的方式进行调整：

TypeError: Index does not support mutable operations

### 3.2. 将Index看作有序集合

`Pandas` 对象被设计用于实现许多操作，如连接（join）数据集，其中会涉及许多集合操作。  
`Index` 对象遵循 `Python` 标准库的集合（`set`）数据结构的许多习惯用法，包括并集、交集、差集等：

In [35]:
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])

# indA & indB  # 交集
indA.intersection(indB)

Int64Index([3, 5, 7], dtype='int64')

In [36]:
# indA | indB  # 并集
indA.union(indB)

Int64Index([1, 2, 3, 5, 7, 9, 11], dtype='int64')

In [37]:
# indA ^ indB  # 异或
indA.symmetric_difference(indB)

Int64Index([1, 2, 9, 11], dtype='int64')